*** MDTI Bulk Lookup ***

This workbook is designed for the data stackers out there.  It will take an array of IPs/Domains and run them through each of these endpoints:

- Hosts
- Reputation 
- Web Components
- Trackers
- Cookies
- PDNS 
- Reverse DNS

If you're familiar with MDTI you'll note that some items are missing, this is just due to sheer volume of the data returned.  The output of this notebook will add everything into it's own sheet within an Excel file.  The nomenclatures and coding blocks used are in no way a best practice, this notebook should serve as a jumping off point for those defenders who need a way to get started with MDTI.

Download the Excel template file from here: 

If you haven't yet setup your application, go here: https://techcommunity.microsoft.com/t5/microsoft-defender-threat/what-s-new-apis-in-microsoft-graph/ba-p/3780350

I would suggest adding the blog site to favorites: https://techcommunity.microsoft.com/t5/microsoft-defender-threat/bg-p/DefenderThreatIntelligence

%pip install pandas, msal, openpyxl

In [ ]:
# For this notebook your env will need pandas, openpyxl, msal installed via pip or from source if you prefer.  The client_secret.txt file and the empty IOC xlsx files should be in the same folder the notebook is running from or should have a specified filepath.

import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
import os
import requests
import json

# Read the client secret from a text file
with open("client_secret.txt", "r") as f:
    client_secret = f.read().strip()

# Set the client secret as an environment variable
os.environ["CLIENT_SECRET"] = client_secret

from msal import ConfidentialClientApplication

# Azure AD application credentials
client_id = ""
# If you are not using a secure string or key vault, you will need to un-comment the line below and add the secret there.
# client_secret = ""
tenant_id = ""


# Create a ConfidentialClientApplication object
app = ConfidentialClientApplication(
    client_id=client_id,
    client_credential=client_secret,
    authority=f"https://login.microsoftonline.com/{tenant_id}",
)

# Get a token from Azure AD
result = None
scopes = ["https://graph.microsoft.com/.default"]
result = app.acquire_token_silent(scopes=scopes, account=None)

if not result:
    result = app.acquire_token_for_client(scopes=scopes)

# Get the access token
access_token = result["access_token"]

# Print the access token
print("Access Token:", access_token)
 


In [3]:
suspect_iocs = ["64.52.80.63","5.252.177.180","193.149.189.224","64.190.113.172","64.52.80.209","65.109.31.190","45.128.156.46","84.252.94.184","192.240.116.106","5.255.100.206","168.100.10.226","45.61.139.183","162.33.178.162","64.190.113.53","162.33.178.34","64.52.80.206"]
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}


for ip in suspect_iocs:
    workbook = load_workbook(filename='MDTIIOC.xlsx')
    ws = workbook['Host Info']
    count = 0
    for row in ws:
        if not all([cell.value is None for cell in row]):
            count += 1
    services = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)
    responseS = requests.get(services, headers=headers)
    dataS = responseS.json()
    dataframeS = pd.DataFrame(dataS)
    with pd.ExcelWriter('MDTIIOC.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        dataframeS.to_excel(writer, sheet_name='Host Info', startrow=count+1)
                
for ip in suspect_iocs:
    workbook = load_workbook(filename='MDTIIOC.xlsx')
    ws = workbook['Reputation']
    count = 0
    for row in ws:
        if not all([cell.value is None for cell in row]):
            count += 1
    reputation = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/reputation"
    responseR = requests.get(reputation, headers=headers)
    dataR = responseR.json()
    dataframeR = pd.DataFrame(dataR)
    dataframeR.drop(dataframeR.loc[dataframeR.index==id].index)        
    with pd.ExcelWriter('MDTIIOC.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer: 
        dataframeR.to_excel(writer, sheet_name='Reputation', startrow=count+1)
            
            
for ip in suspect_iocs:
    workbook = load_workbook(filename='MDTIIOC.xlsx')
    ws = workbook['Components']
    count = 0
    for row in ws:
        if not all([cell.value is None for cell in row]):
            count += 1
    comps = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/components"
    responseC = requests.get(comps, headers=headers)
    dataC = responseC.json()
    dataframeC = pd.DataFrame(dataC)['value'].apply(pd.Series)
    dataframeC.drop(dataframeC.loc[dataframeC.index==id].index)       
    with pd.ExcelWriter('MDTIIOC.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer: 
        dataframeC.to_excel(writer, sheet_name='Components', startrow=count+1)
            
            
for ip in suspect_iocs:
    workbook = load_workbook(filename='MDTIIOC.xlsx')
    ws = workbook['Trackers']
    count = 0
    for row in ws:
        if not all([cell.value is None for cell in row]):
            count += 1
    trackers = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/trackers?count=true"
    responseT = requests.get(trackers, headers=headers)
    dataT = responseT.json()
    dataframeT = pd.DataFrame(dataT)['value'].apply(pd.Series)
    dataframeT.drop(dataframeT.loc[dataframeT.index==id].index)     
    with pd.ExcelWriter('MDTIIOC.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        dataframeT.to_excel(writer, sheet_name='Trackers', startrow=count+1)
            
       
        
for ip in suspect_iocs:
    workbook = load_workbook(filename='MDTIIOC.xlsx')
    ws = workbook['Cookies']
    count = 0
    for row in ws:
        if not all([cell.value is None for cell in row]):
            count += 1
    cookies = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/cookies"
    responseCO = requests.get(cookies, headers=headers)
    dataCO = responseCO.json()
    dataframeCO = pd.DataFrame(dataCO)['value'].apply(pd.Series)
    dataframeCO.drop(dataframeCO.loc[dataframeCO.index==id].index)
    with pd.ExcelWriter('MDTIIOC.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        dataframeCO.to_excel(writer, sheet_name='Cookies', startrow=count+1)


for ip in suspect_iocs:
    workbook = load_workbook(filename='MDTIIOC.xlsx')
    ws = workbook['PDNS']
    count = 0
    for row in ws:
        if not all([cell.value is None for cell in row]):
            count += 1
    services = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/passivedns"
    responsePDNS = requests.get(services, headers=headers)
    dataPDNS = responsePDNS.json()
    dataframePDNS = pd.json_normalize(dataPDNS['value'])
    with pd.ExcelWriter('MDTIIOC.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        dataframePDNS.to_excel(writer, sheet_name='PDNS', startrow=count+1)

for ip in suspect_iocs:
    workbook = load_workbook(filename='MDTIIOC.xlsx')
    ws = workbook['ReverseDNS']
    count = 0
    for row in ws:
        if not all([cell.value is None for cell in row]):
            count += 1
    services = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/"+(ip)+"/passivednsreverse"
    responseRDNS = requests.get(services, headers=headers)
    dataRDNS = responseRDNS.json()
    dataframeRDNS = pd.json_normalize(dataRDNS['value'])
    with pd.ExcelWriter('MDTIIOC.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        dataframePDNS.to_excel(writer, sheet_name='ReverseDNS', startrow=count+1)
